In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import datasets
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, recall_score, precision_score, f1_score
import missingno as msno
import pgeocode
import xgboost as xgb
from sklearn.preprocessing import TargetEncoder, LabelEncoder
import scipy.stats as stats
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import re

In [2]:
df = pd.read_csv("datacleaned.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 30 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   LoanNr_ChkDgt       899164 non-null  int64  
 1   Name                899150 non-null  object 
 2   City                899164 non-null  object 
 3   State               899164 non-null  object 
 4   Zip                 899164 non-null  int64  
 5   Bank                897605 non-null  object 
 6   BankState           897598 non-null  object 
 7   NAICS               899164 non-null  int64  
 8   ApprovalDate        899164 non-null  object 
 9   ApprovalFY          899164 non-null  int64  
 10  Term                899164 non-null  int64  
 11  NoEmp               899164 non-null  int64  
 12  NewExist            899028 non-null  float64
 13  CreateJob           899164 non-null  int64  
 14  RetainedJob         899164 non-null  int64  
 15  FranchiseCode       899164 non-nul

# selection des données



liste drop
data = df.drop(["LoanNr_ChkDgt", "MIS_Status", "ChgOffDate", "DisbursementDate", "DisbursementGross", "BalanceGross", "ChgOffprinGr", "SBA_Appv", "Name", "City", "Zip", "Bank", "BankState", "NAICS", "RevLineCr", "ApprovalDate"])


In [11]:
data = df.drop(["LoanNr_ChkDgt", "MIS_Status", "domaine d’activite","FranchiseCode", "ChgOffDate", "DisbursementDate", "DisbursementGross", "BalanceGross", "ChgOffPrinGr", "SBA_Appv", "Name", "City", "Zip", "Bank", "BankState", "NAICS", "RevLineCr", "ApprovalDate"], axis=1)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   State        899164 non-null  object 
 1   ApprovalFY   899164 non-null  int64  
 2   Term         899164 non-null  int64  
 3   NoEmp        899164 non-null  int64  
 4   NewExist     899028 non-null  float64
 5   CreateJob    899164 non-null  int64  
 6   RetainedJob  899164 non-null  int64  
 7   UrbanRural   899164 non-null  int64  
 8   LowDoc       899164 non-null  object 
 9   GrAppv       899164 non-null  float64
 10  activities   899164 non-null  object 
 11  Franchise    899164 non-null  object 
dtypes: float64(2), int64(6), object(4)
memory usage: 82.3+ MB


# Creation Pipelines


Preprocessing
utilisé un robust scaler

In [18]:
X = data
y = df["MIS_Status"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
numeric_features = ["ApprovalFY", "Term", "NoEmp", "NewExist", "CreateJob", "RetainedJob", "UrbanRural", "GrAppv"]
categorical_features = ["State", "LowDoc", "activities", "Franchise"]
Target_features = ["MIS_Status"]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

Target_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

In [20]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('target', Target_transformer, Target_features)
    ])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000)),
])

In [21]:
pipeline.fit(X_train, y_train)


/home/flavigny/Documents/DEV_IA/AccordPretEntreprise/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['ApprovalFY', 'Term',
                                                   'NoEmp', 'NewExist',
                                                   'CreateJob', 'RetainedJob',
                                                   'UrbanRural', 'GrAppv']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['State', 'LowDoc',
                                                   'activities',
                                                   'Franchise'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [22]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

      CHGOFF       0.70      0.27      0.39     32413
       P I F       0.86      0.97      0.91    147420

    accuracy                           0.85    179833
   macro avg       0.78      0.62      0.65    179833
weighted avg       0.83      0.85      0.82    179833



reequilibrage avec Smote

In [24]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#  caractéristiques numériques et catégoriques
numeric_features = ["ApprovalFY", "Term", "NoEmp", "NewExist", "CreateJob", "RetainedJob", "UrbanRural", "GrAppv"]
categorical_features = ["State", "LowDoc", "activities", "Franchise"]
#Target_features = ["MIS_Status"]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

Target_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        #
    ])

# prétraitement
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

#  SMOTE 
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_transformed, y_train)

#
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_resampled, y_train_resampled)


y_pred = classifier.predict(X_test_transformed)
print(classification_report(y_test, y_pred))


/home/flavigny/Documents/DEV_IA/AccordPretEntreprise/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

      CHGOFF       0.37      0.81      0.51     32413
       P I F       0.94      0.70      0.80    147420

    accuracy                           0.72    179833
   macro avg       0.66      0.76      0.66    179833
weighted avg       0.84      0.72      0.75    179833



# AUC ROC

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
import numpy as np


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# features numériques et catégoriques
numeric_features = ["ApprovalFY", "Term", "NoEmp", "NewExist", "CreateJob", "RetainedJob", "UrbanRural", "GrAppv"]
categorical_features = ["State", "LowDoc", "activities", "Franchise"]
Target_features = ["MIS_Status"]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
])

Target_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('target', Target_transformer, Target_features)
    ])

#  prétraitement
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# SMOTE 
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_transformed, y_train)

# Entraîner le modèle 
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_resampled, y_train_resampled)


y_pred = classifier.predict(X_test_transformed)

# probabilités 
y_scores = classifier.predict_proba(X_test_transformed)[:, 1]

#étiquettes en valeurs binaires
y_test_binary = np.where(y_test == 'P I F', 1, 0)

# AUC-ROC
auc_roc = roc_auc_score(y_test_binary, y_scores)

# courbe ROC
fpr, tpr, _ = roc_curve(y_test_binary, y_scores)


print("AUC-ROC:", auc_roc)

# rapport de classification
print(classification_report(y_test_binary, y_pred))


In [13]:
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr,tpr)

plt.figure()
plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.xlabel("Taux de FP")
plt.ylabel("Taux de TP")
plt.title("Courbe ROC")
plt.show()
from sklearn.metrics import roc_curve, auc

ValueError: y_true takes value in {'CHGOFF', 'P I F'} and pos_label is not specified: either make y_true take value in {0, 1} or {-1, 1} or pass pos_label explicitly.